# Truy cập vào thư mục Colab trên Google Drive thông qua tài khoản email cá nhân.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Cài đặt các thư viện 

In [0]:
!pip install tensorflow
!pip install keras
!pip install pillow

In [3]:
!ls


drive  sample_data


In [0]:
!ls


# Truy cập vào thư mục Colab Notebooks

In [30]:
cd Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [0]:
!ls

In [0]:
!wget https://farm5.staticflickr.com/4091/4994221690_d070e8a355_b_d.jpg

--2019-06-28 07:36:01--  https://farm5.staticflickr.com/4091/4994221690_d070e8a355_b_d.jpg
Resolving farm5.staticflickr.com (farm5.staticflickr.com)... 13.32.80.5, 2600:9000:2004:ce00:0:5a51:64c9:c681, 2600:9000:2004:c400:0:5a51:64c9:c681, ...
Connecting to farm5.staticflickr.com (farm5.staticflickr.com)|13.32.80.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100640 (98K) [image/jpeg]
Saving to: ‘4994221690_d070e8a355_b_d.jpg.1’

4994221690_d070e8a3 100%[===================>]  98.28K  --.-KB/s    in 0.05s   

2019-06-28 07:36:01 (1.92 MB/s) - ‘4994221690_d070e8a355_b_d.jpg.1’ saved [100640/100640]



In [0]:
# Di chuyển đến Thư mục Colab Notebooks chứa code có trong Thư mục My Drive

# Truy cập thư viện OS. 
# Sau đó mount các thư mục con trong Colab Notebooks.

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

In [0]:
!ls

In [9]:
# Cài đặt face_recognition
!pip3 install mtcnn pytesseract face_recognition

     |████████████████████████████████| 2.3MB 40.2MB/s 
     |████████████████████████████████| 174kB 43.4MB/s 
     |████████████████████████████████| 100.2MB 298kB/s 
  Stored in directory: /root/.cache/pip/wheels/85/81/65/6363fa5aafd7a155c896591e0c7c6e27b69642aa82b9cbf076
  Stored in directory: /root/.cache/pip/wheels/cd/4a/30/998e01b892300ba0ccce7b806b6e889794605a384dac81a49a
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built mtcnn pytesseract face-recognition-models


In [33]:
#RUN FROM HERE
import math
from sklearn import neighbors
import os
import os.path
import pickle
from PIL import Image, ImageDraw
import cv2
from pathlib import Path
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

# Định nghĩa hàm train

def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    """
    Trains a k-nearest neighbors classifier for face recognition.
    :param train_dir: directory that contains a sub-directory for each known person, with its name.
     (View in source code to see train_dir example tree structure)
     Structure:
        <train_dir>/
        ├── <person1>/
        │   ├── <somename1>.jpeg
        │   ├── <somename2>.jpeg
        │   ├── ...
        ├── <person2>/
        │   ├── <somename1>.jpeg
        │   └── <somename2>.jpeg
        └── ...
    :param model_save_path: (optional) path to save model on disk
    :param n_neighbors: (optional) number of neighbors to weigh in classification. Chosen automatically if not specified
    :param knn_algo: (optional) underlying data structure to support knn.default is ball_tree
    :param verbose: verbosity of training
    :return: returns knn classifier that was trained on the given data.
    """
    X = []
    y = []
    # Loop through each person in the training set
    # Quet các thư mục chứa các ảnh trong tập DATATRAIN
    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue

        # Loop through each training image for the current person
        # Quet các file trong thư mục con và lấy các ra các khuôn mặt trả về face trong các tấm ảnh.
        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image)

            if len(face_bounding_boxes) != 1:
                # If there are no people (or too many people) in a training image, skip the image.
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                # Add face encoding for current image to the training set
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)

    # Determine how many neighbors to use for weighting in the KNN classifier
    # Chọn K người láng giềng gần nhất
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    # Create and train the KNN classifier
    # Tạo
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    # Save the trained KNN classifier
    # Lưu file vào thư mục model_save_path
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

# Định nghĩa hàm nhận dạng khuôn mặt => đưa vào 1 tấm ảnh và trả về kết quả.
def predict(X_img_path, knn_clf=None, model_path=None, distance_threshold=0.75):
    """
    Recognizes faces in given image using a trained KNN classifier
    :param X_img_path: path to image to be recognized
    :param knn_clf: (optional) a knn classifier object. if not specified, model_save_path must be specified.
    :param model_path: (optional) path to a pickled knn classifier. if not specified, model_save_path must be knn_clf.
    :param distance_threshold: (optional) distance threshold for face classification. the larger it is, the more chance
           of mis-classifying an unknown person as a known one.
    :return: a list of names and face locations for the recognized faces in the image: [(name, bounding box), ...].
        For faces of unrecognized persons, the name 'unknown' will be returned.
    """
    if not os.path.isfile(X_img_path) or os.path.splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        raise Exception("Invalid image path: {}".format(X_img_path))

    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")

    # Load a trained KNN model (if one was passed in)
    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)

    # Load image file and find face locations
    # Lấy khuôn mặt từ ảnh test
    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)

    # If no faces are found in the image, return an empty result.
    if len(X_face_locations) == 0:
        return []

    # Find encodings for faces in the test iamge
    # Rút trích đặc trưng khuôn mặt
    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    # Use the KNN model to find the best matches for the test face
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    # Predict classes and remove classifications that aren't within the threshold
    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]


def show_prediction_labels_on_image(img_path, predictions):
    """
    Shows the face recognition results visually.
    :param img_path: path to image to be recognized
    :param predictions: results of the predict function
    :return:
    """
    pil_image = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(pil_image)

    for name, (top, right, bottom, left) in predictions:
        # Draw a box around the face using the Pillow module
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        # There's a bug in Pillow where it blows up with non-UTF-8 text
        # when using the default bitmap font
        name = name.encode("UTF-8")

        # Draw a label with a name below the face
        text_width, text_height = draw.textsize(name)
        text_width = text_width
        text_height = text_height
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 8), name, fill=(255, 255, 255, 255))
        #draw.text((10,60), name, fill=(255, 255, 255, 255))
        #draw.text((18,60), "World", font=fnt, fill=(255,255,255,255))

    # Remove the drawing library from memory as per the Pillow docs
    del draw

    # Display the resulting image
#     pil_image.imshow(img_path)
    pil_image.show()
    pil_image.save("Output/" + Path(img_path).name)


if __name__ == "__main__":
    # STEP 1: Train the KNN classifier and save it to disk
    # Once the model is trained and saved, you can skip this step next time.
    print("Training KNN classifier...")
    classifier = train("./DATATRAIN", model_save_path="trained_knn_model.clf", n_neighbors=3)
    print("Training complete!")

    # STEP 2: Using the trained classifier, make predictions for unknown images
    for image_file in os.listdir("DataImage"):
        full_file_path = os.path.join("DataImage", image_file)

        #print("Looking for faces in {}".format(image_file))

        # Find all people in the image using a trained classifier model
        # Note: You can pass in either a classifier file name or a classifier model instance
        predictions = predict(full_file_path, model_path="trained_knn_model.clf")

        # Print results on the console
        #for name, (top, right, bottom, left) in predictions:
        #    print("- Found {} at ({}, {})".format(name, left, top))

        # Display results overlaid on an image
        show_prediction_labels_on_image(os.path.join("DataImage", image_file), predictions)



Training KNN classifier...
Training complete!


# Nhận xét kết  quả (--Đếm bằng tay--)
# IMG_5328: Tỉ lệ nhận diện 23% (nhận diện đúng 3 gương mặt/13 gương mặt ).
# IMG_5329: Tỉ lệ nhận diện  36% (nhận diện đúng 4 gương mặt/11 gương mặt).
# IMG_5330: Tỉ lệ nhận diện  50% (nhận diện đúng 6 gương mặt/12 gương mặt).
# IMG_5331: Tỉ lệ nhận diện  11% (nhận diện đúng 1 gương mặt/9 gương mặt).
# IMG_5332: Tỉ lệ nhận diện  57% (nhận diện đúng 4 gương mặt/7 gương mặt).
# Kết quả nhận diện thấp, dữ liệu dùng để train khá ít, hình ảnh train mờ không rõ mặt, không nhận diện được các khuôn mặt nghiêng, che khuất hơn 2/3 khuôn mặt.


# Link tham khảo:
# https://github.com/ageitgey/face_recognition
